## 读入文本  
我们用一部英文小说，即H. G. Well的Time Machine，作为示例，展示文本预处理的具体过程。

In [11]:
import collections
import re

def read_time_machine():
    with open('./timemachine.txt', 'r', encoding='utf-8') as f:
        print(f)
        lines = [re.sub('[^a-z]+',' ',line.strip().lower()) for line in f]
    return lines

lines = read_time_machine()
lines[0:10]

<_io.TextIOWrapper name='./timemachine.txt' mode='r' encoding='utf-8'>


['the time machine',
 '',
 '',
 'an invention',
 '',
 '',
 '',
 '',
 'by h g wells',
 '']

## 分词

In [14]:
def tokenize(sentences, token='word'):
    if token == 'word':
        return [sentence.split(' ') for sentence in sentences]
    elif tiken == 'char':
        return [list(sentence) for sentence in sentences]
    else:
        print('ERROR : unkown token type' + token)
tokens = tokenize(lines)
tokens[0:10]

[['the', 'time', 'machine'],
 [''],
 [''],
 ['an', 'invention'],
 [''],
 [''],
 [''],
 [''],
 ['by', 'h', 'g', 'wells'],
 ['']]

## 建立字典  
为了方便模型处理，我们需要将字符串转换为数字。因此我们需要先构建一个字典（vocabulary），将每个词映射到一个唯一的索引编号。

In [18]:
class Vocab(object):
    def __init__(self, tokens, min_freq=0, use_special_tokens=False):
        counter = count_corpus(tokens)  # : 
        self.token_freqs = list(counter.items())
        self.idx_to_token = []
        if use_special_tokens:
            # padding, begin of sentence, end of sentence, unknown
            self.pad, self.bos, self.eos, self.unk = (0, 1, 2, 3)
            self.idx_to_token += ['', '', '', '']
        else:
            self.unk = 0
            self.idx_to_token += ['']
        self.idx_to_token += [token for token, freq in self.token_freqs
                        if freq >= min_freq and token not in self.idx_to_token] #去除低频词汇和特殊词
        self.token_to_idx = dict()
        for idx, token in enumerate(self.idx_to_token):
            self.token_to_idx[token] = idx

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]
    
    
def count_corpus(sentences):
    tokens = [tk for st in sentences for tk in st]
    return collections.Counter(tokens)  # 返回一个字典，记录每个词的出现次数

In [19]:
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[0:10])

[('', 0), ('the', 1), ('time', 2), ('machine', 3), ('an', 4), ('invention', 5), ('by', 6), ('h', 7), ('g', 8), ('wells', 9)]


## 现有工具分词  
spacy和NLTK

In [2]:
text = "Mr. Chen doesn't agree with my suggestion."

In [3]:
import spacy
import en_core_web_sm
nlp = spacy.load('en_core_web_sm')
doc = nlp(text)
print([token.text for token in doc])


['Mr.', 'Chen', 'does', "n't", 'agree', 'with', 'my', 'suggestion', '.']


In [5]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download()
from nltk import data
data.path.append('./')
print(word_tokenize(text))

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\Admin/nltk_data'
    - 'c:\\Python37\\nltk_data'
    - 'c:\\Python37\\share\\nltk_data'
    - 'c:\\Python37\\lib\\nltk_data'
    - 'C:\\Users\\Admin\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - './'
    - ''
**********************************************************************
